## Excel “Filter and Edit” - Demonstrated in Pandas

我從很多人那裡聽說，我以前寫的關於pandas中常見的Excel任務的文章，對於幫助新的pandas用戶將Excel過程轉化為等價的pandas代碼很有幫助。本文將延續這一傳統，以Excel的Filter函數為模型，說明各種pandas索引的過程。

大多數新的pandas用戶最先學習的東西之一是基本的數據過濾。儘管在過去的幾個月裡，我一直在使用pandas，但最近我意識到，pandas的過濾方法還有一個好處，我在日常工作中沒有使用。也就是說，你可以對給定的一組列進行過濾，但使用簡化的pandas語法更新另一組列。這類似於我在Excel中所說的 "過濾和編輯 "過程。

本文將介紹一些過濾pandas DataFrame和根據不同標準更新數據的例子。在此過程中，我將進一步解釋一些關於panda的索引，以及如何使用.loc和.iloc等索引方法，根據簡單或複雜的標準，快速、輕鬆地更新數據子集。

### Excel。 "過濾和編輯"

在數據透視表之外，Excel中最常用的工具之一是過濾器。這個簡單的工具允許用戶通過各種數字、文本和格式化標準快速過濾和排序數據。下面是一些樣本數據的基本截圖，其中有按幾個不同標準過濾的數據。過濾過程很直觀，即使是最初級的Excel用戶也很容易掌握。我還注意到，人們會用這個功能來選擇數據行，然後根據行的標準來更新其他列。下面的例子顯示了我所描述的情況。

在這個例子中，我已經根據賬號、SKU和單價過濾了數據。然後我手動添加了一個佣金率列，並在每個單元格中鍵入了0.01。這種方法的好處是，它很容易理解，可以幫助人們管理相對複雜的數據，而不需要寫很長的Excel公式或進入VBA。這種方法的缺點是不可重複，外界的人可能很難理解任何過濾器使用了哪些標準。

例如，如果你看一下下面的截圖，如果不看每一列，就沒有明顯的辦法知道哪些是被過濾的。幸運的是，我們可以在pandas中做一些非常類似的事情。毫不奇怪，在pandas中很容易用簡單乾淨的代碼來執行這個 "過濾和編輯 "模型。

### Boolean Indexing 布林索引
現在你對這個問題有了一個感覺，我想通過一些細節來了解pandas中的布林索引。如果你想在最廣泛的意義上理解pandas的數據索引和選擇，這是一個重要的概念。這個概念對於新的pandas用戶來說可能有點複雜（對於有經驗的用戶來說可能太基本了），但我認為花點時間理解它是很重要的。如果你掌握了這個概念，在pandas中處理數據的基本過程就會更加簡單明了。

Pandas支持通過使用標籤、基於位置的整數或布林值（真/假）的列表來建立索引（或選擇數據）。使用布林值列表來選擇一行被稱為布林索引，這將是本文的重點。

我發現，我的pandas工作流程往往主要集中在使用布林值列表來選擇我的數據。換句話說，當我創建pandas DataFrame時，我傾向於在DataFrame中保留默認索引。因此，索引本身並沒有真正的意義，對於選擇數據也不直接。

關鍵點
布林索引是在pandas中選擇數據行的強大而有用的方法之一（有幾種）。
讓我們看看一些例子的DataFrames來幫助澄清pandas中的布林索引的作用。

首先，我們將創建一個非常小的DataFrame，純粹來自於一個python列表，並使用它來展示布林索引是如何工作的。

In [2]:
import pandas as pd
sales = [('account', ['Jones LLC', 'Alpha Co', 'Blue Inc', 'Mega Corp']),
         ('Total Sales', [150, 200, 75, 300]),
         ('Country', ['US', 'UK', 'US', 'US'])]
#df = pd.DataFrame.from_dict(sales) # Row
df = pd.DataFrame.from_dict(dict(sales)) 
df

,account,Total Sales,Country
0,Jones LLC,150,US
1,Alpha Co,200,UK
2,Blue Inc,75,US
3,Mega Corp,300,US


注意到0-3的值是如何被自動分配到行中的嗎？這些就是索引，它們在這個數據集中並沒有什麼特別的意義，但是對於pandas來說是很有用的，對於下面沒有描述的其他用例來說，理解這些索引很重要。當我們提到布爾索引時，我們只是指我們可以傳入一個代表我們想要查看的每一行的真或假值的列表。

在這個例子中，如果我們想查看Jones LLC、Blue Inc和Mega Corp的數據，我們可以看到True False列表會是這樣的。

In [3]:
indices = [True, False, True, True]
df[indices]

,account,Total Sales,Country
0,Jones LLC,150,US
2,Blue Inc,75,US
3,Mega Corp,300,US


這種手動創建索引的方式是可行的，但顯然無法擴展，也無法對瑣碎的數據集有多大作用。幸運的是，pandas使得使用簡單的查詢語言來創建這些布爾索引變得非常容易，使用過python（或任何語言）的人應該對這些語言很熟悉。

舉個例子，讓我們看一下美國的所有銷售線。如果我們執行一個基於國家列的python表達式。

In [4]:
df.Country == 'US'

0     True
1    False
2     True
3     True
Name: Country, dtype: bool

這個例子展示了pandas如何將你的傳統python邏輯，應用到DataFrame中，並返回一個布爾值的列表。然後，這個布爾值列表可以被傳遞給DataFrame以獲得相應的數據行。

在真正的代碼中，你不會做這個兩步過程。這樣做的速記方法通常看起來像這樣。

In [5]:
df[df["Country"] == 'US']

,account,Total Sales,Country
0,Jones LLC,150,US
2,Blue Inc,75,US
3,Mega Corp,300,US


In [6]:
filter = df["Country"] == 'US'
df[filter]

,account,Total Sales,Country
0,Jones LLC,150,US
2,Blue Inc,75,US
3,Mega Corp,300,US


雖然這個概念很簡單，但你可以利用python的力量編寫相當複雜的邏輯來過濾你的數據。

關鍵點
在這個例子中，df[df.Country == 'US'] 相當於df[df["Country"] == 'US'] '.'符號比較乾淨，但是當列名中有空格的時候就不適用了。

### Selecting the Columns選擇列
現在我們已經知道瞭如何選擇數據行，那麼我們如何控制顯示哪些列呢？在上面的例子中，沒有明顯的方法可以做到這一點。 Pandas可以使用三種基於位置的索引來支持這種用例：.loc , iloc 。這些函數也允許我們在迄今為止所看到的行選擇之外選擇列。

關於何時使用.loc、iloc或.ix，有很多混淆之處。區別的快速總結是：
- .loc是用於標籤索引的
- .iloc用於基於位置的整數。

因此，問題是，我應該使用哪一個？我得承認，我在這個問題上也被絆倒過幾次。我發現我最常使用的是.loc。主要是因為我的數據並不適合基於位置的索引（換句話說，我很少發現自己需要.iloc），所以我堅持使用.loc。

公平地說，這些方法中的每一種都有它們的位置，在許多情況下都很有用。特別是在處理多索引數據框架的時候。我不會在本文中討論這個話題--也許會在以後的文章中討論。繼續我們的例子，如果我們只想顯示與我們的索引相對應的賬戶名，該怎麼辦？使用.loc就很簡單。

In [8]:
df.loc[[True, True, False, True], "account"]

0    Jones LLC
1     Alpha Co
3    Mega Corp
Name: account, dtype: object

In [9]:
df.loc[[True, True, False, True], ["account", "Country"]]

,account,Country
0,Jones LLC,US
1,Alpha Co,UK
3,Mega Corp,US


這個過程可以被認為有點相當於我們上面討論的Excel的過濾器。你有一個額外的好處，就是你也可以限制你檢索的列數，而不僅僅是行數。

In [10]:
df.loc[df["Total Sales"] > 200, ["account", "Country"]]

,account,Country
3,Mega Corp,US


### Editing Columns 編輯列
所有這些都是很好的背景，但是當你使用類似的方法來更新一個或多個基於行選擇的列時，這個過程才真正發揮了作用。舉個簡單的例子，讓我們給我們的數據添加一個佣金率列。

比方說，如果你賣出了100多個，你的費率就是5%。基本過程是設置一個布爾索引來選擇列，然後將值分配給費率列。


In [13]:
df["rate"] = 0.02  # add
df

,account,Total Sales,Country,rate
0,Jones LLC,150,US,0.02
1,Alpha Co,200,UK,0.02
2,Blue Inc,75,US,0.02
3,Mega Corp,300,US,0.02


### Bringing It All Together 將一切結合起來

在最後一個例子中，我們將使用以下規則創建一個簡單的佣金計算器。
- 所有的佣金都在交易層面上計算
- 所有銷售的基本佣金為2%。
- 所有襯衫的佣金為2.5%。
- 正在進行一項特別計劃，在一次交易中銷售超過10條腰帶可獲得4%的佣金
- 在一次交易中，所有鞋類銷售>1000美元，有250美元的特別獎金和4.5%的佣金。

為了在Excel中做到這一點，使用過濾和編輯的方法。

- 添加一個2%的佣金列
- 添加一個0美元的獎金欄
- 對襯衫進行過濾，並將其價值改為2.5%。
- 清除過濾器
- 過濾皮帶和數量>10，並將其值改為4%。
- 清除過濾器
- 篩選出鞋子>1000美元，並添加佣金和獎金值分別為4.5%和250美元

這種方法在Excel中操作起來很簡單，但它的可重複性和可審計性都不強。當然，在Excel中還有其他方法可以完成這個任務--比如公式或VBA。然而，這種 "過濾和編輯 "的方法是很常見的，也是對pandas邏輯的說明。

現在，讓我們在pandas中走完整個例子，讀入Excel文件並添加一列2%的默認費率。

In [15]:
import pandas as pd
df = pd.read_excel("data/sample-sales-reps.xlsx")

,account number,customer name,sales rep,sku,category,quantity,unit price,ext price,date,commission
0,680916,Mueller and Sons,Loring Predovic,GP-14407,Belt,19,88.49,1681.31,2015-11-17 05:58:34,0.02
1,680916,Mueller and Sons,Loring Predovic,FI-01804,Shirt,3,78.07,234.21,2016-02-13 04:04:11,0.02
2,530925,Purdy and Sons,Teagan O'Keefe,EO-54210,Shirt,19,30.21,573.99,2015-08-11 12:44:38,0.02
3,14406,"Harber, Lubowitz and Fahey",Esequiel Schinner,NZ-99565,Shirt,12,90.29,1083.48,2016-01-23 02:15:50,0.02
4,398620,Brekke Ltd,Esequiel Schinner,NZ-99565,Shirt,5,72.64,363.20,2015-08-10 07:16:03,0.02


In [18]:
df["commission"] = .02
df["bonus"] = 0
df.loc[df["category"] == "Shirt", ["commission"]] = .025
df.loc[(df["category"] == "Belt") & (df["quantity"] >= 10), ["commission"]] = .04
df.loc[(df["category"] == "Shoes") & (df["ext price"] >= 1000 ), ["bonus", "commission"]] = 250, 0.045

# Show data
df.iloc[3:8]

,account number,customer name,sales rep,sku,category,quantity,unit price,ext price,date,commission,bonus
0,680916,Mueller and Sons,Loring Predovic,GP-14407,Belt,19,88.49,1681.31,2015-11-17 05:58:34,0.040,0
1,680916,Mueller and Sons,Loring Predovic,FI-01804,Shirt,3,78.07,234.21,2016-02-13 04:04:11,0.025,0
2,530925,Purdy and Sons,Teagan O'Keefe,EO-54210,Shirt,19,30.21,573.99,2015-08-11 12:44:38,0.025,0
3,14406,"Harber, Lubowitz and Fahey",Esequiel Schinner,NZ-99565,Shirt,12,90.29,1083.48,2016-01-23 02:15:50,0.025,0
4,398620,Brekke Ltd,Esequiel Schinner,NZ-99565,Shirt,5,72.64,363.20,2015-08-10 07:16:03,0.025,0


In [22]:
#  Calculate the compensation for each row
df["comp"] = df["commission"] * df["ext price"] + df["bonus"]

# Summarize and round the results by sales rep
df.groupby(["sales rep"])["comp"].sum().round(2)

sales rep
Ansley Cummings       2169.76
Beth Skiles           3028.60
Esequiel Schinner    10451.21
Loring Predovic      10108.60
Shannen Hudson        5275.66
Teagan O'Keefe        7989.52
Trish Deckow          5807.74
Name: comp, dtype: float64